# Installing and Importing the required Libraries

In [125]:
import pandas as pd # library for data analysis
import numpy as np
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
import json # library to handle JSON files

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Scraping the Wikipedia page for the table of postal codes of Canada and Data preprocessing and cleaning

In [126]:
# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)


# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
postaltable=soup.find('table',{'class':"wikitable"})


df=pd.read_html(str(postaltable))
# convert list to dataframe
df=pd.DataFrame(df[0])
print(df.head())


# Dropping the rows where Borough is 'Not assigned'
df_new = df[df.Borough != 'Not assigned']

# Combining the neighbourhoods with same Postalcode

df2= df_new.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df2.head()

# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
df2.Neighbourhood =  np.where(df2.Neighbourhood == "Not assigned",df2.Borough,df2.Neighbourhood)
df2

df2.shape





200
  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


(103, 3)

# Importing the csv file conatining the latitudes and longitudes for various neighbourhoods in Canada

In [127]:
# get the data and create the dataframe



df_geo = pd.read_csv('https://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [128]:
df3 = pd.merge(df2, df_geo, on = 'Postal Code')
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [129]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df3['Borough'].unique()),
        df3.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [130]:
Toronto_data = df3[df3['Borough'] == 'Toronto'].reset_index(drop=True)
Toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude


In [131]:
#We work with only "Borough" which contain the word Toronto 
df4=df3[df3['Borough'].str.contains('Toronto')]
Toronto_data=df4.reset_index(drop=True)
Toronto_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [132]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


# Visualizing all the Neighbourhoods of the above data frame using Folium

In [133]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighbourhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [134]:




CLIENT_ID = 'ZNJ23MZU50ERHJGOART4DVUC0ONLCPWVWCWUPZRFHWU4IG52'
 # your Foursquare ID
CLIENT_SECRET = 'NXAM4XVP3XYVPH3FUHZ5MKRC2DBXMALRQDYCJQVOZDXKFZPK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


radius = 500 # define radius

Your credentails:
CLIENT_ID: ZNJ23MZU50ERHJGOART4DVUC0ONLCPWVWCWUPZRFHWU4IG52
CLIENT_SECRET:NXAM4XVP3XYVPH3FUHZ5MKRC2DBXMALRQDYCJQVOZDXKFZPK


In [135]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [136]:
Toronto_data_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville


KeyError: 'groups'

In [ ]:
print(Toronto_data_venues.shape)
Toronto_data_venues.head()

In [ ]:
#Number of venues per neighborhood
Toronto_data_venues.groupby('Neighborhood').count()

In [ ]:
#Number of unique venue categories
print('There are {} uniques categories.'.format(len(Toronto_data_venues['Venue Category'].unique())))

In [ ]:
#print out the list of categories
Toronto_data_venues['Venue Category'].unique()

In [ ]:
"Chinese Restaurant" in Toronto_data_venues['Venue Category'].unique()

In [ ]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_data_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
print(Toronto_onehot.shape)

Toronto_onehot.columns

In [ ]:
Toronto_onehot.head()

In [ ]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(Toronto_grouped.shape)
Toronto_grouped

# Neighborhoods vs No of Chinese Restaurant

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

graph = pd.DataFrame(Toronto_onehot.groupby('Neighborhood')['Chinese Restaurant'].sum())
graph = graph.sort_values(by ='Chinese Restaurant', ascending=False)
graph.iloc[:14].plot(kind='bar', figsize=(10,6))
plt.xlabel("Neighborhoods")
plt.ylabel("No of Chinese Restaurant")
plt.title("Neighborhoods vs No of Chinese Restaurant")
plt.show()

In [ ]:
len(Toronto_grouped[Toronto_grouped["Chinese Restaurant"] > 0])

In [ ]:
Toronto_asian = Toronto_grouped[["Neighborhood","Chinese Restaurant"]]

In [ ]:
Toronto_asian

In [ ]:
# set number of clusters
kclusters = 2

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:38] 

In [ ]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Toronto_merged = Toronto_asian.copy()

# add clustering labels
Toronto_merged["Cluster Labels"] = kmeans.labels_

In [ ]:
Toronto_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
Toronto_merged.head()

In [ ]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Toronto_data_venues.set_index("Neighborhood"), on="Neighborhood")

print(Toronto_merged.shape)
Toronto_merged.head()

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Neighborhood Latitude'], Toronto_merged['Neighborhood Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
#Cluster 0
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0]

In [ ]:
#Cluster 1
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1]

In [ ]:
Toronto_merged['Cluster Labels'].value_counts()